# Zania | AI Challenge

## Problem Statement
Create an AI agent that leverages the capabilities of a large language model. This agent should be able to extract answers based on the content of a large PDF document and post the results on Slack. Ideally, you use OpenAI LLMs. You can also use the Langchain or LLama Index framework to implement this agentic functionality.



In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [37]:
# os.environ.get('OPENAI_API_KEY')

In [38]:
# os.environ.get('SLACK_USER_TOKEN')

## Load pdf file


In [5]:
# Transform Loaders.
def load_document(file):
    import os
    name, extension = os.path.splitext(file)
    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    else:
        print('Document format is not supported!!')
        return None
    
    data = loader.load()
    return data


###  Chunking
- `Chunking is the process of breaking down large pieces of text into smaller segments`.

In [6]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap= 0)
    chunks = text_splitter.split_documents(data)
    return chunks

### Text embedding
- We'll be using Openai's text embedding ADA 002, which has a cost.


In [7]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total token: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 100 * 0.0004:.6f}')

-----


## Using Chroma as a Vector DB 

In [31]:
# !pip install -q chromadb

- Now defining a new function that creates the embeddings.

- Using the OpenAI embeddings class, saves them in a database and returns the database.



In [8]:
def create_embedding_chroma(chunks, persist_dir='./chroma.db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings(
        model= 'text-embedding-3-small',
        dimensions = 1536
    )    
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_dir)
    return vector_store
    

def load_embedding_chroma(persist_dir='./chroma.db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(
        model= 'text-embedding-3-small',
        dimensions = 1536
    )    
    vector_store = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

    return vector_store

In [8]:
data = load_document('handbook.pdf')
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embedding_chroma(chunks)

Loading handbook.pdf


### Check the cost 

In [22]:
print_embedding_cost(chunks)

Total token: 27505
Embedding Cost in USD: 0.110020


In [28]:
# load data
vector_store = load_embedding_chroma()

In [29]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

## Using a Custom prompt
- Let's explore how to change the system prompt and use prompt engineering techniques to ask questions in specific ways.

In [40]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

memory =ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

system_template = r'''
Use the following pieces of context to answer the user's question.
If you don't know the answer in the provided context, just respond "Data Not Available"
-------------
Context:```{context}```
'''

user_template = '''
Question: ```{question}```
Chat History: ```{chat_history}```
'''

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

# Chain type stuff means use all of the text from the documents.
crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    chain_type='stuff',
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
    verbose=False
)

In [41]:
print(qa_prompt)

input_variables=['chat_history', 'context', 'question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nUse the following pieces of context to answer the user\'s question.\nIf you don\'t know the answer in the provided context, just respond "Data Not Available"\n-------------\nContext:```{context}```\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='\nQuestion: ```{question}```\nChat History: ```{chat_history}```\n'))]


In [42]:
def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

In [23]:
# db = load_embedding_chroma()

In [43]:
q = "What is the name of the company?"
result = ask_question(q, crc)
print(result['answer'])

Zania, Inc.


In [32]:
q = "Who is the CEO of the company?"
result = ask_question(q, crc)
print(result['answer'])

{
    "answer": "Shruti Gupta"
}


### Here we can ask multiple question

In [33]:
import time
i = 1
print('While Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i += 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye')
        time.sleep(2)
        break
        
    result = ask_question(q, crc)
    answer = result['answer']
    print(f'\nAnswer: {answer}')
    print(f'\n{"-" * 50} \n')

While Quit or Exit to quit.
Question #1: What is the name of the company?

Answer: {
    "answer": "Zania, Inc."
}

-------------------------------------------------- 

Question #2: Who is the CEO of the company?

Answer: ```json
{
    "answer": "Shruti Gupta"
}
```

-------------------------------------------------- 

Question #3: What is their vacation policy?

Answer: ```json
{
    "answer": "PTO may be used for vacation, sick time, or other personal matters."
}
```

-------------------------------------------------- 

Question #4: What is the termination policy?

Answer: ```json
{
    "answer": "Violation of attendance policy or job abandonment may result in disciplinary action, up to and including termination of employment. Violation of other policies may also lead to termination of employment."
}
```

-------------------------------------------------- 

Question #5: exit
Quitting ... bye bye


Thank you

----

### With slack integration

In [34]:
# !pip install --upgrade --quiet  slack_sdk > /dev/null

In [16]:
from langchain_community.agent_toolkits import SlackToolkit
toolkit = SlackToolkit()
tools = toolkit.get_tools()
tools

[SlackGetChannel(client=<slack_sdk.web.client.WebClient object at 0x11e753a50>),
 SlackGetMessage(client=<slack_sdk.web.client.WebClient object at 0x11e7508d0>),
 SlackScheduleMessage(client=<slack_sdk.web.client.WebClient object at 0x11ead6450>),
 SlackSendMessage(client=<slack_sdk.web.client.WebClient object at 0x11ead7910>)]

In [17]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [34]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    prompt=prompt,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [35]:
def ask_question_send_to_slack(q, chain):
    result = chain.invoke({'question': q})
    json = {
        'question': result['question'],
        'answer': result['answer']
    }
    agent_executor.invoke({
        "input": f"Send {json} in the #gen-ai channel. Note use `channel` as key of channel id, and `message` as key of content to sent in the channel."
    })
    return json

In [36]:
q = "What is the name of the company?"
result = ask_question_send_to_slack(q, crc)
print(result)

{'question': 'What is the name of the company?', 'answer': 'Zania, Inc.'}
